# Usage and more of the Calibrations Class [v1.1]

In [1]:
# imports
import os
import glob

from importlib import reload

import numpy as np

from pypit import arparse
from pypit import calibrations
from pypit import pypitsetup

from pypit import arpixels

from pypit.spectrographs import keck_lris

In [2]:
os.getenv('PYPIT_DEV')

'/home/xavier/local/Python/PYPIT-development-suite/'

## Multi-Slit Example

In [3]:
spectrograph='keck_lris_red'
spectro_class = keck_lris.KeckLRISBSpectrograph()

## Fuss with settings..

In [4]:
from pypit import arparse as settings
settings.dummy_settings(spectrograph=spectrograph)

[INFO]    :: arparse.py 75 load_file() - Loading default settings
[INFO]    :: arparse.py 87 load_file() - Loading base settings from settings.baseargflag
[INFO]    :: arparse.py 1393 run_ncpus() - Setting 7 CPUs
[INFO]    :: arparse.py 75 load_file() - Loading default settings
[INFO]    :: arparse.py 75 load_file() - Loading default settings
[INFO]    :: arparse.py 87 load_file() - Loading base settings from settings.basespect
[INFO]    :: arparse.py 75 load_file() - Loading default settings


## FITS table

In [5]:
keck_lris_files = glob.glob(os.getenv('PYPIT_DEV')+'RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR*')
keck_lris_files.sort()
len(keck_lris_files)

23

In [6]:
keck_lris_files[4]

'/home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.07348.fits'

In [7]:
# Pop the unknown file
keck_lris_files.pop(4)

'/home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.07348.fits'

In [8]:
setupc = pypitsetup.PypitSetup(settings.argflag, settings.spect)

In [9]:
_, fitstbl, setup_dict = setupc.run(file_list=keck_lris_files)

[INFO]    :: arload.py 192 load_headers() - Successfully loaded headers for file:
             /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.05529.fits
[INFO]    :: arload.py 192 load_headers() - Successfully loaded headers for file:
             /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.05589.fits
[INFO]    :: arload.py 192 load_headers() - Successfully loaded headers for file:
             /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.05649.fits
[WARNING] :: arload.py 157 load_headers() - TARGNAME keyword not in header. Setting to None
[WARNING] :: arload.py 157 load_headers() - RA keyword not in header. Setting to None
[INFO]    :: arload.py 192 load_headers() - Successfully loaded headers for file:
             /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20

In [10]:
fitstbl[0:5]

directory,filename,utc,target,idname,time,date,equinox,ra,dec,airmass,naxis0,naxis1,binning,exptime,filter1,filter2,hatch,shutopen,shutclose,decker,lamps,slitwid,slitlen,detrot,dichroic,dispname,dispangle,lampname01,lampstat01,lampname02,lampstat02,lampname03,lampstat03,lampname04,lampstat04,lampname05,lampstat05,lampname06,lampstat06,lampname07,lampstat07,lampname08,lampstat08,lampname09,lampstat09,lampname10,lampstat10,lampname11,lampstat11,lampname12,lampstat12,wavecen,instrume,arc,bias,dark,pinhole,pixelflat,science,standard,trace,unknown,failures,sci_ID
str92,str22,str11,str14,str4,float64,str19,str4,str11,str11,float64,str4,str4,str3,int64,str5,str4,str6,str4,str4,str8,str4,str4,str4,str4,str3,str8,float64,str7,str3,str4,str3,str5,str3,str7,str3,str4,str3,str7,str3,str5,str3,str7,str3,str9,str3,str6,str3,str6,str3,str7,str3,float64,str13,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,int64
/home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/,LR.20160216.05529.fits,01:32:09.90,,dark,1378417.5361,2016-02-16T01:32:11,None,00:00:00.00,+00:00:00.0,1.41294028,None,None,"2,2",1,clear,None,closed,None,None,long_0.7,None,None,None,None,560,600/7500,27.0812645,MERCURY,on,NEON,on,ARGON,on,CADMIUM,on,ZINC,on,KRYPTON,on,XENON,on,FEARGON,off,DEUTERIUM,off,FLAMP1,off,FLAMP2,off,HALOGEN,off,7150.0078125,keck_lris_red,True,False,False,False,False,False,False,False,False,False,0
/home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/,LR.20160216.05589.fits,01:33:09.40,,dark,1378417.55263,2016-02-16T01:33:11,None,00:00:00.00,+00:00:00.0,1.41294028,None,None,"2,2",1,clear,None,closed,None,None,long_0.7,None,None,None,None,560,600/7500,27.0812645,MERCURY,on,NEON,on,ARGON,on,CADMIUM,on,ZINC,on,KRYPTON,on,XENON,on,FEARGON,off,DEUTERIUM,off,FLAMP1,off,FLAMP2,off,HALOGEN,off,7150.0078125,keck_lris_red,True,False,False,False,False,False,False,False,False,False,0
/home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/,LR.20160216.05649.fits,01:34:09.64,,dark,1378417.56936,2016-02-16T01:34:11,None,00:00:00.00,+00:00:00.0,1.41294028,None,None,"2,2",1,clear,None,closed,None,None,long_0.7,None,None,None,None,560,600/7500,27.0812645,MERCURY,on,NEON,on,ARGON,on,CADMIUM,on,ZINC,on,KRYPTON,on,XENON,on,FEARGON,off,DEUTERIUM,off,FLAMP1,off,FLAMP2,off,HALOGEN,off,7150.0078125,keck_lris_red,True,False,False,False,False,False,False,False,False,False,0
/home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/,LR.20160216.05709.fits,01:35:09.44,None,dark,1378417.58597,2016-02-16T01:35:11,None,None,+00:00:00.0,1.41294028,None,None,"2,2",1,clear,None,closed,None,None,long_0.7,None,None,None,None,560,600/7500,27.0812645,MERCURY,on,NEON,on,ARGON,on,CADMIUM,on,ZINC,on,KRYPTON,on,XENON,on,FEARGON,off,DEUTERIUM,off,FLAMP1,off,FLAMP2,off,HALOGEN,off,7150.0078125,keck_lris_red,True,False,False,False,False,False,False,False,False,False,1
/home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/,LR.20160216.07412.fits,02:03:32.09,unknown,dark,1378418.05894,2016-02-16T02:03:31,None,15:20:00.00,+45:00:00.0,1.41309745,None,None,"2,2",0,clear,None,closed,None,None,long_0.7,None,None,None,None,560,600/7500,27.0812645,MERCURY,off,NEON,off,ARGON,off,CADMIUM,off,ZINC,off,KRYPTON,off,XENON,off,FEARGON,off,DEUTERIUM,off,FLAMP1,off,FLAMP2,off,HALOGEN,off,7150.0078125,keck_lris_red,False,True,False,False,False,False,False,False,False,False,0


## Choose the detector, sci_ID

In [11]:
det = 2
sci_ID = 1
setup = 'C_02_aa'

##  Calibrations

### More settings fussing

In [12]:
# TODO -- Update/avoid the following with new settings
tsettings = settings.argflag.copy()
tsettings['detector'] = settings.spect[settings.get_dnum(det)]
try:
    tsettings['detector']['dataext'] = tsettings['detector']['dataext01']  # Kludge; goofy named key
except KeyError: # LRIS, DEIMOS
    tsettings['detector']['dataext'] = None
tsettings['detector']['dispaxis'] = settings.argflag['trace']['dispersion']['direction']
tsettings['detector']['binning'] = fitstbl['binning'][0]

# New ones
tsettings['masters'] = settings.argflag['reduce']['masters']
tsettings['masters']['directory'] = settings.argflag['run']['directory']['master']+'_'+ settings.argflag['run']['spectrograph']
tsettings['masters']['setup'] = setup

## Instantiate

In [13]:
reload(calibrations)
caliBrate = calibrations.MultiSlitCalibrations(fitstbl, save_masters=False, write_qa=False)

In [14]:
# Setup for a specific science frame + detector
caliBrate.reset(setup, det, sci_ID, tsettings)

## Datasec_img

In [15]:
_ = caliBrate.get_datasec_img()

[INFO]    :: keck_lris.py 236 read_lris() - Reading LRIS file: /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.40478.fits


## Bias

In [16]:
bias = caliBrate.get_bias()

[INFO]    :: keck_lris.py 236 read_lris() - Reading LRIS file: /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.07703.fits
[INFO]    :: keck_lris.py 236 read_lris() - Reading LRIS file: /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.07762.fits
[INFO]    :: keck_lris.py 236 read_lris() - Reading LRIS file: /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.07820.fits
[INFO]    :: keck_lris.py 236 read_lris() - Reading LRIS file: /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.07878.fits
[INFO]    :: keck_lris.py 236 read_lris() - Reading LRIS file: /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.07937.fits
[INFO]    :: keck_lris.py 236 read_lris() - Reading LRIS file: /home/xavier/local/Python/PYPIT-development-sui

In [17]:
caliBrate.show(bias)

## Grab the bias again

In [18]:
bias = caliBrate.get_bias()

[INFO]    :: calibrations.py 191 get_bias() - Reloading the bias from the internal dict


In [19]:
caliBrate.calib_dict[caliBrate.setup].keys()

dict_keys(['bias'])

## Try to jump too far ahead

In [20]:
caliBrate.get_wv_calib()

[WARNING] :: calibrations.py 588 _chk_objs() - You need to generate msarc prior to this calibration..
[WARNING] :: calibrations.py 597 _chk_objs() - Use get_arc


## Run them all 'from scratch'

In [21]:
caliBrate.steps

['datasec_img',
 'bias',
 'arc',
 'bpm',
 'pixlocn',
 'slits',
 'wv_calib',
 'tilts',
 'pixflatnrm',
 'wave']

In [22]:
caliBrate.run_the_steps()

[INFO]    :: keck_lris.py 236 read_lris() - Reading LRIS file: /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.40478.fits
[INFO]    :: calibrations.py 191 get_bias() - Reloading the bias from the internal dict
[INFO]    :: calibrations.py 164 get_arc() - Preparing a master arc frame
[INFO]    :: keck_lris.py 236 read_lris() - Reading LRIS file: /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.05709.fits
[INFO]    :: keck_lris.py 236 read_lris() - Reading LRIS file: /home/xavier/local/Python/PYPIT-development-suite/RAW_DATA/Keck_LRIS_red/long_600_7500_d560/LR.20160216.05709.fits
[INFO]    :: processimages.py 263 bias_subtract() - Bias subtracting your image(s)
[INFO]    :: arprocimg.py 74 bias_subtract() - Subtracting bias image from raw frame
[INFO]    :: keck_lris.py 160 bpm() - Using hard-coded BPM for det=2 on LRISr
[INFO]    :: arpixels.py 65 core_gen_pixloc() - Deriving p

---------------------------------------------------
Report:
::   Number of lines recovered    = 83
::   Number of lines analyzed     = 83
::   Number of acceptable matches = 81
::   Best central wavelength      = 7174.84A
::   Best dispersion              = 1.59706A/pix
::   Best solution used pix_tol   = 1.0
::   Best solution had unknown    = True
---------------------------------------------------


[INFO]    :: ararc.py 113 get_censpec() - Extracting an approximate arc spectrum at the centre of each slit
[WORK IN ]::
[PROGRESS]:: artracewave.py 205 trace_tilt() - Detecting lines for slit 1
[INFO]    :: ararc.py 206 detect_lines() - Detecting lines
[INFO]    :: ararc.py 217 detect_lines() - Detecting the strongest, nonsaturated lines
[INFO]    :: artracewave.py 264 trace_tilt() - Modelling arc line tilts with 51 arc lines
[WORK IN ]::
[PROGRESS]:: artracewave.py 270 trace_tilt() - This next step could be multiprocessed to speed up the reduction
[INFO]    :: artracewave.py 444 trace_tilt() - Completed spectral tilt tracing
[WARNING] :: wavetilts.py 192 _analyze_lines() - There were 13 additional arc lines that should have been traced
             (perhaps lines were saturated?). Check the spectral tilt solution
[INFO]    :: artracewave.py 778 fit_tilts() - Fitting tilts with a low order, 2D legendre
[WORK IN ]::
[PROGRESS]:: arutils.py 754 polyfit2d_general() - Generalize to differ

In [23]:
caliBrate.show(caliBrate.mswave)